# Render trained appearance networks
## Init

In [1]:
import sys
sys.path.insert(0, '/mnt/qb/work/bethge/ahochlehnert48/code/continual-nerf/inerf/inerf')
import typing
import torch
import numpy as np
import numpy.typing as npt

from nerf_helpers import load_nerf, NeRFAppearance
from render_helpers import render, to8b, get_rays
from scipy.spatial.transform import Rotation as R
import imageio
from tqdm import tqdm

import os
import json
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from dataclasses import dataclass

In [2]:
BASE_DIR = '/mnt/qb/work/bethge/ahochlehnert48/code/continual-nerf/inerf'

@dataclass
class Args:
    netdepth=8
    netwidth=256
    netdepth_fine=8
    netwidth_fine=256
    chunk=1024*32
    netchunk=1024*64

    model_name='model_080000'
    model_fine_name='model_fine_080000'
    # data_dir = './data/nerf_synthetic/'
    data_dir = os.path.join(BASE_DIR, 'data/shapenet/chairs_latent/1031fc859dc3177a2f84cb7932f866fd')
    ckpt_dir = os.path.join(BASE_DIR, 'nerf/logs/chairs_latent/1031fc859dc3177a2f84cb7932f866fd')
    dataset_type='blender'
    obs_img_num=1
    
    use_viewdirs=True
    N_importance=64
    N_samples=64
    white_bkgd=True
    
    i_embed=0
    perturb=0.
    raw_noise_std=0.
    lindisp=True
    multires=10
    multires_views=4

    # sample_count=10_000
    sample_count=250

    appearance_dim=3


args = Args()

### Load NeRF

In [3]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
H, W = 400, 400


with open(os.path.join(args.data_dir, 'transforms_test.json'), 'r') as fp:
    meta = json.load(fp)
frames = meta['frames']

# camera_angle_x = 0.691111147403717
camera_angle_x = meta['camera_angle_x']
print('camera_angle_x:', camera_angle_x)
focal = .5 * W / np.tan(.5 * camera_angle_x)

camera_angle_x: 0.6911112070083618


In [4]:
render_kwargs = load_nerf(args, device, NeRFAppearance)
bds_dict = {
    'near': 2.,
    'far': 9.,
}
render_kwargs.update(bds_dict)

Found keras weights.
Reloading from /mnt/qb/work/bethge/ahochlehnert48/code/continual-nerf/inerf/nerf/logs/chairs_latent/1031fc859dc3177a2f84cb7932f866fd/model_080000.npy /mnt/qb/work/bethge/ahochlehnert48/code/continual-nerf/inerf/nerf/logs/chairs_latent/1031fc859dc3177a2f84cb7932f866fd/model_fine_080000.npy
Not ndc!


### Appearance latents

In [5]:
render_kwargs['network_fn'].appearance_latents

tensor([[ 0.4753,  1.0021,  0.0517],
        [-0.0047, -0.0843,  0.6260],
        [ 0.8409,  0.4505,  1.1615],
        [ 1.0958,  0.9010,  0.5823],
        [ 0.0603,  1.2047, -0.0070],
        [ 0.8243,  0.8157,  0.8099],
        [ 0.2583, -0.2460,  0.4570],
        [ 0.3120, -0.4154,  0.2860],
        [-0.1711,  0.6088,  0.8479],
        [ 1.5499,  0.2115,  0.7491],
        [-0.0557,  0.3692,  0.8664],
        [ 0.7776,  0.7380,  0.4524],
        [ 0.3913,  0.6509,  0.0258],
        [ 0.7095,  0.5817,  0.2781],
        [ 0.2023,  1.2767,  0.3565],
        [-0.0930,  0.6458, -0.1877],
        [ 0.1779, -0.1380,  0.5932],
        [ 0.4855,  1.1941, -0.2068],
        [-0.1362, -0.1328,  0.3789],
        [ 0.6889,  0.9539, -0.2104],
        [ 0.5552,  0.5888, -0.3360],
        [-0.0463,  1.5378,  0.4262],
        [-0.0480,  0.3634,  0.9397],
        [ 0.4887,  1.0414,  0.2709],
        [ 0.7131,  0.5793,  0.1604],
        [ 0.6983,  0.5994, -0.0729],
        [-0.0564,  0.1222,  0.8267],
 

## Render appearance latents with fixed pose

In [14]:
import matplotlib.animation as animation

# ani.save('movie.mp4')
# plt.show()

print(f'Render {len(render_kwargs["network_fn"].appearance_latents)} images')
imgs = []
frame = frames[10]
pose = torch.from_numpy(np.array(frame['transform_matrix'], dtype=np.float32)).cuda()
rays_o, rays_d = get_rays(H, W, focal, pose)  # (H, W, 3), (H, W, 3)
batch_rays = torch.stack([rays_o, rays_d], 0)
for al in tqdm(render_kwargs["network_fn"].appearance_latents[:10]):
    render_kwargs['appearance_latent'] = al.cuda()
    rgb, disp, acc, extras = render(H, W, focal, chunk=args.chunk, rays=batch_rays,
                                    verbose=0 < 10, retraw=True,
                                    **render_kwargs)
    rgb = rgb.cpu().detach().numpy()
    rgb8 = to8b(rgb)
    # filename = os.path.join(SAVE_DIR, f'{frame["file_path"]}.png')
    # imageio.imwrite(filename, rgb8)
    imgs.append(rgb8)

# fig = plt.figure()
# pls = [plt.imshow(img) for img in imgs]
# ani = animation.ArtistAnimation(fig, pls, interval=50, blit=True,
#                                 repeat_delay=1000)
# plt.show()
imageio.mimwrite('APPEARACNCE_video.gif', imgs, fps=1)

Render 100 images


100%|██████████| 10/10 [00:57<00:00,  5.79s/it]


## Render test poses with foxed appearance latent

In [16]:
print(f'Render {len(render_kwargs["network_fn"].appearance_latents)} images')
imgs = []
render_kwargs['appearance_latent'] = render_kwargs["network_fn"].appearance_latents[5].cuda()
for frame in tqdm(frames):
    pose = torch.from_numpy(np.array(frame['transform_matrix'], dtype=np.float32)).cuda()
    rays_o, rays_d = get_rays(H, W, focal, pose)  # (H, W, 3), (H, W, 3)
    batch_rays = torch.stack([rays_o, rays_d], 0)
    rgb, disp, acc, extras = render(H, W, focal, chunk=args.chunk, rays=batch_rays,
                                    verbose=0 < 10, retraw=True,
                                    **render_kwargs)
    rgb = rgb.cpu().detach().numpy()
    rgb8 = to8b(rgb)
    imgs.append(rgb8)

imageio.mimwrite('APPEARACNCE2_video.gif', imgs, fps=8)

Render 100 images


  0%|          | 0/100 [00:00<?, ?it/s]